<a href="https://colab.research.google.com/github/mikmikthemimic/FasterRCNN_Baseline_Resnet50/blob/main/Models_To_Compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#clone here
!git clone https://github.com/mikmikthemimic/FasterRCNN_Baseline_Resnet50.git

Cloning into 'FasterRCNN_Baseline_Resnet50'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 143 (delta 55), reused 117 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (143/143), 7.54 MiB | 12.14 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
%%capture
import os

if not os.path.isdir('/content/FasterRCNN_Baseline_Resnet50/pytorch_faster_rcnn_tutorial/data/heads/input/'):
  !unzip '/content/drive/MyDrive/raw dataset/fin_dataset.zip' -d '/content/FasterRCNN_Baseline_Resnet50/pytorch_faster_rcnn_tutorial/data/heads'

In [ ]:
%cd /content/FasterRCNN_Baseline_Resnet50

/content/FasterRCNN_Baseline_Resnet50


In [ ]:
!pip install -q -r requirements_no-hash.txt
#!pip install -q -r requirements_colab.txt

ERROR: Could not find a version that satisfies the requirement pywin32 (from versions: none)
ERROR: No matching distribution found for pywin32


In [ ]:
!pip install albumentations==1.2.1

In [ ]:
from dataclasses import asdict, dataclass, field
from typing import Dict, List, Optional, Tuple

from pytorch_faster_rcnn_tutorial.backbone_resnet import ResNetBackbones

@dataclass
class Parameters:
    """
    Dataclass for the parameters.
    """

    BATCH_SIZE: int = 18
    CACHE: bool = False
    SAVE_DIR: Optional[
        str
    ] = None  # checkpoints will be saved to cwd (current working directory) if None
    LOG_MODEL: bool = True  # whether to log the model to neptune after training
    ACCELERATOR: Optional[str] = "cuda"  # set to "gpu" if you want to use GPU \\ cuda
    # Available names are: auto, cpu, cuda, mps, tpu
    LR: float = 0.0025
    PRECISION: int = 32
    CLASSES: int = 6
    SEED: int = 42
    MAXEPOCHS: int = 12  # 4
    FOLDS: int = 5 # 5
    PATIENCE: int = 50
    #BACKBONE: ResNetBackbones = ResNetBackbones.RESNET34
    FPN: bool = False        # defailt ni faster rcnn is true
    ANCHOR_SIZE: Tuple[Tuple[int, ...], ...] = ((32,), (64,), (128,), (256,), (512,))
    # ANCHOR_SIZE: Tuple[Tuple[int, ...], ...] = ((8, 16, 32), (32, 64), (32, 64, 128, 256, 512), (200, 300))
    # ANCHOR_SIZE: Tuple[Tuple[int, ...], ...] = ((32,64,128,256,512),)
    ASPECT_RATIOS: Tuple[Tuple[float, ...]] = ((0.5, 1.0, 2.0),)
    MIN_SIZE: int = 1024
    MAX_SIZE: int = 1025
    IMG_MEAN: List = field(default_factory=lambda: [0.485, 0.456, 0.406])
    IMG_STD: List = field(default_factory=lambda: [0.229, 0.224, 0.225])
    IOU_THRESHOLD: float = 0.6
    FAST_DEV_RUN: bool = False

    def __post_init__(self):
        if self.SAVE_DIR is None:
            self.SAVE_DIR: str = str(pathlib.Path.cwd())

In [ ]:
from pydantic import BaseSettings, Field

class NeptuneSettings(BaseSettings):
    """
    Reads the variables from the environment.
    Errors will be raised if the required variables are not set.
    """

    api_key: str = Field(default=..., env="NEPTUNE")
    OWNER: str = "mikmikthemimic"  # set your name here, e.g. mikmikthemimic
    PROJECT: str = "GM-Thesis3"  # set your project name here, e.g. GM-Thesis3
    EXPERIMENT: str = "GM-Thesis3"  # set your experiment name here, e.g. GM-Thesis3

    class Config:
        # this tells pydantic to read the variables from the .env file
        env_file = ".env"

In [ ]:
import pathlib
from lightning.pytorch import seed_everything

ROOT_PATH: pathlib.Path = pathlib.Path.cwd()

# environment variables (pydantic BaseSettings class)
neptune_settings: NeptuneSettings = NeptuneSettings()

# parameters (dataclass)
parameters: Parameters = Parameters()

# data path relative to this file (pathlib)
data_path: pathlib.Path = (
    ROOT_PATH / "pytorch_faster_rcnn_tutorial" / "data" / "heads"
)

# random seed (function that sets seed for pseudo-random number generators in: pytorch, numpy, python.random)
seed_everything(parameters.SEED)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

In [ ]:
%load_ext autoreload
%autoreload 2

from pytorch_faster_rcnn_tutorial.faster_RCNN import (
    FasterRCNNLightning,
    get_faster_rcnn_resnet,
    get_faster_rcnn_vgg
)


from torchvision.models.detection.faster_rcnn import FasterRCNN
import torch
from torchvision.ops import MultiScaleRoIAlign

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model init
#print(parameters.BACKBONE)
model_init: FasterRCNN = get_faster_rcnn_vgg(
    num_classes=parameters.CLASSES,
    pretrained=True,
    min_size=parameters.MIN_SIZE,
    max_size=parameters.MAX_SIZE,
).to(device)


# lightning model
model: FasterRCNNLightning = FasterRCNNLightning(
    model=model_init, lr=parameters.LR, iou_threshold=parameters.IOU_THRESHOLD
).to(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [ ]:
import logging
import sys

import albumentations
import numpy as np

from lightning.pytorch import Trainer
# from lightning.pytorch import seed_everything
from lightning.pytorch.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
    Callback,
)

from lightning.pytorch.loggers import NeptuneLogger
import torch
from torch.utils.data import DataLoader

from pytorch_faster_rcnn_tutorial.datasets import ObjectDetectionDataSet
from pytorch_faster_rcnn_tutorial.faster_RCNN import (
    FasterRCNNLightning,
    get_faster_rcnn_resnet,
)
from pytorch_faster_rcnn_tutorial.transformations import (
    AlbumentationWrapper,
    Clip,
    ComposeDouble,
    FunctionWrapperDouble,
    normalize_01,
)
from pytorch_faster_rcnn_tutorial.utils import (
    collate_double,
    get_filenames_of_path,
    log_model_neptune,
)

from pytorch_faster_rcnn_tutorial.pl_crossvalidate.KFoldTrainer import (
    KFoldTrainer as Trainer,
    KFoldDataModule
)

In [ ]:
logger: logging.Logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d:%(funcName)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

In [ ]:
import time
import os
from lightning.pytorch.callbacks import Callback # Import the Callback class


os.environ['TZ'] = 'Asia/Manila'
time.tzset()
class DurationCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        print(f'Epoch {trainer.current_epoch} | Training started at {time.strftime("%x %X")}')

    def on_validation_start(self, trainer, pl_module):
        self.validation_start_time = time.time()
        print(f'Epoch {trainer.current_epoch}: Validation started at {time.strftime("%x %X")}')

    def on_validation_end(self, trainer, pl_module):
        print(f'Epoch {trainer.current_epoch}: Validation ended at {time.strftime("%x %X")}')
        print(f'Epoch {trainer.current_epoch}: Validation duration: {time.time() - self.validation_start_time:.2f} seconds')

    def on_test_start(self, trainer, pl_module):
        self.test_start_time = time.time()
        print(f'Testing started at {time.strftime("%x %X")}')

    def on_test_end(self, trainer, pl_module):
        print(f'Testing ended at {time.strftime("%x %X")}')
        print(f'Testing duration: {time.time() - self.test_start_time:.2f} seconds')

In [ ]:
# input and target files
inputs: List[pathlib.Path] = get_filenames_of_path(data_path / "input")
targets: List[pathlib.Path] = get_filenames_of_path(data_path / "target")

# test files
tests: List[pathlib.Path] = get_filenames_of_path(data_path / "test")
tests_targets: List[pathlib.Path] = get_filenames_of_path(data_path / "test_targets")

# sort inputs and targets
inputs.sort()
targets.sort()
tests.sort()
tests_targets.sort()

checking = len(inputs) + len(tests)
print(f'Total of the dataset: {checking}')
print(f'Input dataset: {len(inputs)}')
print(f'Target dataset: {len(targets)}')

# training validation test split (80% training, 20% validation)
inputs_train, inputs_valid = inputs[:int(0.8 * len(inputs))], inputs[int(0.8 * len(inputs)):]
targets_train, targets_valid = targets[:int(0.8 * len(targets))], targets[int(0.8 * len(targets)):]

inputs_test = tests
targets_test = tests_targets

total = len(inputs) + len(tests)
print(f'Total of the dataset: {total}')
print(f'Training dataset: {len(inputs_train)} ({len(inputs_train)/total*100:.2f}%)')
print(f'Validation dataset: {len(inputs_valid)} ({len(inputs_valid)/total*100:.2f}%)')
print(f'Test dataset: {len(inputs_test)} ({len(inputs_test)/total*100:.2f}%)')

Total of the dataset: 3200
Input dataset: 2560
Target dataset: 2560
Total of the dataset: 3200
Training dataset: 2048 (64.00%)
Validation dataset: 512 (16.00%)
Test dataset: 640 (20.00%)


In [ ]:
# mapping
mapping: Dict[str, int] = {
    "Vehicle": 1,
    "Pedestrian": 2,
    "Bicycle" : 3,

}

In [ ]:
# training transformations and augmentations
transforms_training: ComposeDouble = ComposeDouble(
    [
        Clip(),
        AlbumentationWrapper(albumentation=albumentations.HorizontalFlip(p=0.5)),
        AlbumentationWrapper(
            albumentation=albumentations.RandomScale(p=0.5, scale_limit=0.5)
        ),
        # AlbuWrapper(albu=A.VerticalFlip(p=0.5)),
        FunctionWrapperDouble(function=np.moveaxis, source=-1, destination=0),
        FunctionWrapperDouble(function=normalize_01),
    ]
)

# validation transformations
transforms_validation: ComposeDouble = ComposeDouble(
    [
        Clip(),
        FunctionWrapperDouble(function=np.moveaxis, source=-1, destination=0),
        FunctionWrapperDouble(function=normalize_01),
    ]
)

# test transformations
transforms_test: ComposeDouble = ComposeDouble(
    [
        Clip(),
        FunctionWrapperDouble(function=np.moveaxis, source=-1, destination=0),
        FunctionWrapperDouble(function=normalize_01),
    ]
)

In [ ]:
# dataset training
dataset_train: ObjectDetectionDataSet = ObjectDetectionDataSet(
    inputs=inputs_train,
    targets=targets_train,
    transform=transforms_training,
    use_cache=parameters.CACHE,
    convert_to_format=None,
    mapping=mapping,
)

# dataset validation
dataset_valid: ObjectDetectionDataSet = ObjectDetectionDataSet(
    inputs=inputs_valid,
    targets=targets_valid,
    transform=transforms_validation,
    use_cache=parameters.CACHE,
    convert_to_format=None,
    mapping=mapping,
)

# dataset test
dataset_test: ObjectDetectionDataSet = ObjectDetectionDataSet(
    inputs=inputs_test,
    targets=targets_test,
    transform=transforms_test,
    use_cache=parameters.CACHE,
    convert_to_format=None,
    mapping=mapping,
)

In [ ]:
if torch.cuda.is_available():
  dev = torch.device("cuda")
  print('CUDA is Available.')
  torch.set_float32_matmul_precision('medium')
  model = model.cuda()

CUDA is Available.


In [ ]:
# dataloader training
dataloader_train: DataLoader = DataLoader(
    dataset=dataset_train,
    batch_size=parameters.BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    persistent_workers=True,
    collate_fn=collate_double,
)

# dataloader validation
dataloader_valid: DataLoader = DataLoader(
    dataset=dataset_valid,
    batch_size=parameters.BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    persistent_workers=True,
    collate_fn=collate_double,
)

# dataloader test
dataloader_test: DataLoader = DataLoader(
    dataset=dataset_test,
    batch_size=2,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_double,
)

In [ ]:
# callbacks
checkpoint_callback: ModelCheckpoint = ModelCheckpoint(
    monitor="Validation_mAP", mode="max", save_top_k=1
)
learning_rate_callback: LearningRateMonitor = LearningRateMonitor(
    logging_interval="step", log_momentum=False
)
early_stopping_callback: EarlyStopping = EarlyStopping(
    monitor="Validation_mAP", patience=parameters.PATIENCE, mode="max"
)

duration_callback: DurationCallback = DurationCallback()

In [ ]:
def get_neptune_logger(id=None) -> NeptuneLogger:
  neptune_logger = None
  if id is None:
    neptune_logger: NeptuneLogger = NeptuneLogger(
        api_key=neptune_settings.api_key,
        project=f"{neptune_settings.OWNER}/{neptune_settings.PROJECT}",  # use your neptune name here
        name=neptune_settings.PROJECT,
        log_model_checkpoints=True,
        capture_hardware_metrics=True,
        capture_stderr=True,
        capture_stdout=True,
    )

    neptune_logger.log_hyperparams(asdict(parameters))
  else:
    neptune_logger: NeptuneLogger = NeptuneLogger(
        api_key=neptune_settings.api_key,
        project=f"{neptune_settings.OWNER}/{neptune_settings.PROJECT}",  # use your neptune name here
        name=neptune_settings.PROJECT,
        log_model_checkpoints=True,
        capture_hardware_metrics=True,
        capture_stderr=True,
        capture_stdout=True,
        with_id=id,
    )

  return neptune_logger

In [ ]:
neptune_logger = get_neptune_logger() # YOU EDIT THIS LINE IF YOU'RE GONNA START FROM A PREVIOUS RUN.
# If resuming from a previous run,
# Pass the id as a string in the function
# i.e. neptune_logger = get_neptune_logger("GMT3-466")
logging.getLogger("neptune").setLevel(logging.CRITICAL)

# Check if there is a saved fold
if neptune_logger.experiment.exists('training/fold'):
    fold = neptune_logger.experiment['training/fold'].fetch()
    fold_index = int(fold) + 1
    print(fold)
    old_model_path = pathlib.Path(ROOT_PATH / "model" / "kfolds" ).as_posix()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if not os.path.exists(old_model_path):
        os.makedirs(old_model_path)
    neptune_logger.experiment[f'artifacts/folds/fold_{fold}'].download(
        destination=old_model_path
    )
    old_model_state = torch.load(old_model_path + f"/fold_{fold}.ckpt", map_location=device)
    old_model_state_keys = {k: v for k, v in old_model_state['state_dict'].items()}    # di nakukuha
    # print("Keys in loaded state dictionary:")
    # for key in old_model_state_keys.keys():
    #     print(key)
    model.load_state_dict(old_model_state_keys)
    print(f"Loaded checkpoint fold {fold_index}")

# trainer init
trainer: Trainer = Trainer(
    accelerator=parameters.ACCELERATOR,
    logger=neptune_logger,
    callbacks=[
        checkpoint_callback,
        learning_rate_callback,
        early_stopping_callback,
        duration_callback
    ],
    default_root_dir=parameters.SAVE_DIR,  # where checkpoints are saved to
    log_every_n_steps=1,  # increase to reduce the amount of log flushes (lowers the overhead)
    num_sanity_val_steps=0,  # set to 0 to skip sanity check
    max_epochs=parameters.MAXEPOCHS,
    fast_dev_run=parameters.FAST_DEV_RUN,  # set to True to test the pipeline with one batch and without validation, testing and logging
    num_folds=parameters.FOLDS,
    shuffle=True,
    stratified=False,
    model_directory= ROOT_PATH / "model" / "kfolds",
)

datamodule = KFoldDataModule(
    num_folds=parameters.FOLDS,    # params.FOLDS    ABI ISET MO TO HA labyu -pastabi :D
    shuffle=True,
    stratified=False,
    train_dataloader=dataloader_train,
    val_dataloaders=dataloader_valid
)

if neptune_logger.experiment.exists('training/fold'):
    datamodule.set_fold_index(fold_index)
    trainer.set_resume_index(fold_index)

datamodule.label_extractor = lambda batch: batch['y']

# start training/testing
cross_val_stats = trainer.cross_validate(
    model=model.to(dev),
    datamodule=datamodule,
)
logger.info(f"Cross validation stats: {cross_val_stats}")

# Create ensemble model
ensemble_model = trainer.create_ensemble(
    model=model.to(dev),
    ckpt_paths=None,
)

#trainer.fit(
#    model=model,
#    train_dataloaders=dataloader_train, #train_loader
#    val_dataloaders=dataloader_valid,
#)

if not parameters.FAST_DEV_RUN:
    trainer.test(ckpt_path="best", dataloaders=dataloader_test)
    # log model
    if parameters.LOG_MODEL:
        checkpoint_path = pathlib.Path(checkpoint_callback.best_model_path)
        save_directory = pathlib.Path(ROOT_PATH / "model")

        log_model_neptune(
            model=model,
            save_directory=save_directory,
            name="best_model.pt",
            neptune_path='artifacts/best_model',
            neptune_logger=neptune_logger,
        )
        log_model_neptune(
            model=ensemble_model,
            save_directory=save_directory,
            name="ensemble_model.pt",
            neptune_path='artifacts/ensemble_model',
            neptune_logger=neptune_logger,
        )

# stop logger
neptune_logger.experiment.stop()
logger.info("Training finished")

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


===== Starting fold 1/5 =====
Resetting model validation steps to empty.


INFO: Restoring states from the checkpoint path at /content/FasterRCNN_Baseline_Resnet50/model/kfolds/kfold_initial_weights.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/FasterRCNN_Baseline_Resnet50/model/kfolds/kfold_initial_weights.ckpt
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:362: The dirpath has changed from None to '/content/FasterRCNN_Baseline_Resnet50/.neptune/GM-Thesis3/GMT3-523/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | FasterRCNN | 43.9 M | train
---------------------------------------------
43.9 M    Trainable

Epoch 0 | Training started at 08/20/24 15:56:36


Training: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 

In [ ]:
torch.save(model.state_dict(), "model.pt")

In [ ]:
torch.save(model, "full_model.pt")

In [ ]:
# prompt: move full_model.pt to google drive

from google.colab import drive
drive.mount('/content/drive')

# Copy the file to Google Drive
!cp full_model.pt /content/drive/MyDrive/baseline_fasterrcnn.pt
!cp model.pt /content/drive/MyDrive/baseline_fasterrcnn_state-dict.pt